In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


 In order to select the 3 albums, the label should be looking at which genres performs the best in USA so now we are going to check that

In [23]:
%%sql
WITH usa AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )

SELECT
    g.name genre,
    count(usa.invoice_line_id) tracks_sold,
    cast(count(usa.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) from usa) percentage_sold
FROM usa
INNER JOIN track t on t.track_id = usa.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC


Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


From this analysis it sees the top genere we should take are:
    1. Punk - Red Tone
    2. Blues - Slim Jim Bites
    3. pop - Meteor and the Girls

Calculating the toal sales amount per each sales rep

In [30]:
%%sql
select * from employee

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2016-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2017-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2017-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [28]:
%%sql
select e.employee_id,
       e.first_name || " " || e.last_name,
       e. title,
       e.hire_date,
       e.country,
       SUM(i.total) as total_sales
FROM employee e
LEFT JOIN customer c on c.support_rep_id = e.employee_id
LEFT JOIN invoice i on i.customer_id = c.customer_id
GROUP BY e.employee_id
ORDER BY total_sales DESC

Done.


employee_id,"e.first_name || "" "" || e.last_name",title,hire_date,country,total_sales
3,Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,Canada,1731.510000000004
4,Margaret Park,Sales Support Agent,2017-05-03 00:00:00,Canada,1584.0000000000032
5,Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,Canada,1393.9200000000028
1,Andrew Adams,General Manager,2016-08-14 00:00:00,Canada,None
2,Nancy Edwards,Sales Manager,2016-05-01 00:00:00,Canada,None
6,Michael Mitchell,IT Manager,2016-10-17 00:00:00,Canada,None
7,Robert King,IT Staff,2017-01-02 00:00:00,Canada,None
8,Laura Callahan,IT Staff,2017-03-04 00:00:00,Canada,None


Looks like the performance is very similar to all 3 sales representatives if we consider their hiring data

Calculate statistics per country:
1. calculate how many customers are per country
2. what is the total value of sales per country
3. Average value of sales per customer per country
4. Average order value per country

In [55]:
%%sql
WITH tot_customers as
(
    SELECT c.country,
           COUNT(DISTINCT c.customer_id) as total_customers,
           CASE
                WHEN COUNT( DISTINCT c.customer_id) = 1 THEN "Other" 
                ELSE c.country
           END as "country_group",
           SUM (i.total) as total_sales,
           COUNT(i.invoice_id) as total_orders 

    FROM customer c
    LEFT JOIN invoice i on i.customer_id = c.customer_id
    group by 1
)
SELECT country_group as country,
       SUM(total_customers) as total_customers,
       sum(total_sales) as total_sales,
       sum(total_orders) as total_orders, 
       sum(total_sales) /  sum(total_customers) as AVG_per_cust,
       sum(total_sales) / sum(total_orders) as AVG_per_order
FROM tot_customers 
GROUP BY 1
order by 5 DESC
       

Done.


country,total_customers,total_sales,total_orders,AVG_per_cust,AVG_per_order
Czech Republic,2,273.23999999999995,30,136.61999999999998,9.107999999999999
Portugal,2,185.13,29,92.565,6.3837931034482756
India,2,183.14999999999995,21,91.57499999999997,8.72142857142857
Brazil,5,427.6799999999999,61,85.53599999999997,7.011147540983605
Germany,4,334.61999999999995,41,83.65499999999999,8.161463414634145
United Kingdom,3,245.52,28,81.84,8.768571428571429
USA,13,1040.4900000000005,131,80.03769230769234,7.942671755725194
France,5,389.07000000000005,50,77.81400000000001,7.781400000000001
Other,15,1094.9399999999998,147,72.996,7.4485714285714275
Canada,8,535.59,76,66.94875,7.047236842105264


According to this analysis it looks like The Czech Republic is the most profitable per customer followed by Portugal and India


Check each invoice to see if it is a full album or individual tracks

In [62]:
%%sql
CREATE VIEW full_albums AS 
SELECT DISTINCT
       al.album_id,
       al.title,
       t.track_id,
       t.name
FROM album al
LEFT JOIN track t on t.album_id = al.album_id
order by 1,3

Done.


[]

In [65]:
%%sql
CREATE VIEW inv_details AS
SELECT i.invoice_id,
       t.album_id,
       al.title,
       t.track_id,
       t.name
FROM invoice i
INNER JOIN invoice_line il on il.invoice_id = i.invoice_id
LEFT JOIN track t on t.track_id = il.track_id
LEFT JOIN album al on al.album_id = t.album_id
ORDER by 1,2,4

Done.


[]

In [75]:
%%sql
CREATE VIEW albums_totals AS
    select 
           f.album_id as id,
           f.title as album_name,
           count(f.track_id) as total_tracks_album
    FROM full_albums f
    group by 1

 

 


Done.


[]

In [77]:
%%sql
CREATE VIEW inv_tracks_totals AS
    select inv.invoice_id as inv_id,
           inv.album_id as album,
           count(inv.track_id) as tot_lines
    FROM inv_details inv
    group by 1
    

Done.


[]

In [94]:
%%sql
select count(inv.inv_id),
      cast( (count(inv.inv_id)) as float) / (select count(inv_id) from inv_tracks_totals) as pct
from inv_tracks_totals inv
Left Join albums_totals alb on alb.id = inv.album
where inv.tot_lines = alb.total_tracks_album
  and alb.total_tracks_album > 5

Done.


count(inv.inv_id),pct
117,0.19055374592833876


IT looks like 19% of purchases are of a full album (for albums with more than 5 songs) so it is not recommended to drop this option
